DMTOOLS External API Test and Demo - 29 July 2025

Notebook to test library that has been published tp pypi test

In [1]:
## need to disconnect and delete the runtime and rerun the notebook to reinstall this properly
## check that it says it has sucessfully installed
!pip install -i https://test.pypi.org/simple/ dmtools-brown-edu --upgrade

Looking in indexes: https://test.pypi.org/simple/, https://www.piwheels.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: dmtools-brown-edu
    Found existing installation: dmtools_brown_edu 0.0.11
    Uninstalling dmtools_brown_edu-0.0.11:
      Successfully uninstalled dmtools_brown_edu-0.0.11


In [6]:
import json
import os

In [7]:
def load_secrets(env_file_path=".env"):
    """
    Loads secrets from a .env file and retrieves 'EMAIL' and 'API_KEY'.

    Args:
        env_file_path (str): Path to the .env file. Defaults to '.env'.

    Returns:
        dict: A dictionary with 'email' and 'api_key' keys.

    Raises:
        FileNotFoundError: If the .env file does not exist.
        KeyError: If either EMAIL or API_KEY is missing.
    """
    if not os.path.exists(env_file_path):
        raise FileNotFoundError(f"{env_file_path} not found.")

    # Load environment variables from the file
    with open(env_file_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                key_value = line.split('=', 1)
                if len(key_value) == 2:
                    key, value = key_value
                    os.environ[key.strip()] = value.strip()

    email = os.getenv('EMAIL')
    api_key = os.getenv('API_KEY')

    if not email or not api_key:
        missing = []
        if not email:
            missing.append('EMAIL')
        if not api_key:
            missing.append('API_KEY')
        raise KeyError(f"Missing required secrets: {', '.join(missing)}")

    return {"email": email, "api_key": api_key}

In [12]:
cwd = os.getcwd()
print("Current working directory:", cwd)

Current working directory: /home/dmtools/Code/dmtools_notebooks/PyPi_Testing


In [8]:
secrets = load_secrets("./secrets/.env")
print(secrets["email"])
print(secrets["api_key"])

FileNotFoundError: ./secrets/.env not found.

In [3]:
from dmtools_brown_edu.dmtools_client_package.dmtools_quart import APIClient

In [7]:
## You need to enter your email address into the colab secrets
## see key in left hand bar
## The library cleans the email address provided
## First time you run this you need to give permission to the notebook to access each of the secrets
#from google.colab import userdata
# email = userdata.get('DMTOOLS_EMAIL')
# print(email)

andrewgaitskell@brown.edu


In [8]:
#api_key = userdata.get('DMTOOLS_APIKEY')

In [ ]:
email = 'andrewgaitskell@brown.edu'
api_key = 

Create the API Client - this connects to the server and allows simpler interactions.

For less verbose calling, you can set suppress_logs=True

In [10]:
Client = APIClient(email, api_key, suppress_logs=False)

Query using key pair - full match on word
Requirement was to exact word match for the key value pair

Example match to CDMS but not to CDMSSuper

2025-01-24 11:37:36,081 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


Below returns a list and the results need to be loaded as json

In [24]:
r = Client.read(query="experiment:CDMS")
record_count = len(r)
experiment_list = []
for rc in range(0, record_count):
  r_p = json.loads(r[rc]['properties'])
  r_p_experiment = r_p['experiment']
  experiment_list.append(r_p_experiment)
unique_list = list(set(experiment_list))
print(unique_list)

2025-01-24 11:58:18,470 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


['CDMS II (Soudan)', 'CDMS I (SUF)']


Read a Single record by the ID
Below returns a json variable in a list - the above result from a query needs to be updated to also return a list of json variables

In [35]:
r_id = Client.read(id=1)
r_id[0]
properties_json = r_id[0]['properties']
#properties_json

2025-01-24 12:08:19,074 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


In [36]:
archived = r_id[0]['archived']
print(archived)

None


In [37]:
archive_via_id = Client.archive(id=1)

2025-01-24 12:08:24,616 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling archive data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling archive data API


In [38]:
archive_via_id

[{'id': 1, 'message': 'Node archived'}, 200]

In [40]:
r_id = Client.read(id=1)
# r_id

2025-01-24 12:09:06,402 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


In [41]:
reverse_archive_via_id = Client.archive_reverse(id=1)

2025-01-24 12:09:16,964 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling remove archive data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling remove archive data API


In [42]:
reverse_archive_via_id

[{'id': 1, 'message': 'Node archival reversed'}, 200]

In [44]:
r_id = Client.read(id=1)
#r_id

2025-01-24 12:09:33,128 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


In [46]:
#new_properties_json = json.loads(r_id)
new_properties = r_id[0]['properties']
type(new_properties)

dict

In [50]:
new_properties_json = new_properties
#new_properties_json

In [51]:
new_data = Client.create(data=new_properties_json,node_type='Data')

2025-01-24 12:18:03,754 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling add data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling add data API


In [52]:
new_data

{'new_record_id': 15737, 'success': True}

In [53]:
new_properties_json['rating']=1000

In [54]:
update_data = Client.update(id=15737,updated_data=new_properties_json,node_type='Data')

2025-01-24 12:18:41,919 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling update data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling update data API


In [55]:
update_data

{'success': True, 'updated_record_id': 15737}

In [56]:
r_id = Client.read(id=15737)
r_id

2025-01-24 12:18:54,243 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


[{'archived': None,
  'created': 'Fri, 24 Jan 2025 12:18:04 GMT',
  'id': 15737,
  'properties': {'comment': ' ',
   'dateAnnouncement': '2003-01-01',
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'red',
   'defaultStyle': 'dash',
   'experiment': 'Theory',
   'greatestHit': 0,
   'label': 'Baer et. al., 2003, constraints on mSUGRA, SI',
   'label_short': 'Baer et. al., 2003, constraints on mSUGRA, SI',
   'measurementType': 'Dir',
   'official': 1,
   'open': 0,
   'public': 1,
   'rating': 1000,
   'raw': '{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;630 3.2e-44;640 3e-44;640 1.2e-44;630 1e-44;225 9e-45;215 7e-45;315 7e-46;470 2.1e-46;650 7e-47;660 6.8e-47;660 2e-47;650 2e-47;470 6e-47;315 1e-46;215 3e-46;100 8e-46;60 1e-45;60 4e-43]}',
   'reference': 'JCAP 0309 (2003) 007 , hep-ph/0305191',
   'resultType': 'Th',
   'spinDependency': 'SI',
   'values': [[['6.00000e+01', '4.00000e-43'],
     ['7.00000e+01', '4.00000e-43'],
     ['1.60000e+02', 

In [57]:
r_id = Client.delete(id=15737)

2025-01-24 12:19:16,014 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling delete data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling delete data API


In [60]:
r_id = Client.read(id=15737)

2025-01-24 12:19:41,484 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


In [62]:
r_id

[]